# Bayesian Hierarchical Poisson model

In [19]:
import pymc as pm
import pandas as pd
import numpy as np
from utils import preprocess

In [20]:
df = pd.read_csv('data/01_input_history.csv')
df.head()


df_train_null, df_train_inactive, df_train_active, df_validation = preprocess.preprocess_ex1(df)

df_train_merged = df_train_active#pd.merge(df_train_active, df_train_inactive, how='outer', on=['unique_id', 'ds', 'Quantity', 'Country', 'Product'])
df_train_static = df_train_merged[['unique_id', 'Country', 'Product']].drop_duplicates().reset_index(drop=True)
df_train_static = pd.get_dummies(df_train_static, columns=['Country', 'Product'], drop_first=True)
assert df_train_static.shape[0] == df_train_merged['unique_id'].nunique(), 'The number of unique_id in static and merged dataframes do not match!'
#df_train_merged = augment_calendar_df(df_train_merged, freq='M')[0]
#df_validation = augment_calendar_df(df_validation, freq='M')[0]
df_train_merged['month'] = df_train_merged['ds'].dt.month
df_validation['month'] = df_validation['ds'].dt.month
df_train_merged = pd.get_dummies(df_train_merged, columns=['month'], drop_first=True)
df_validation = pd.get_dummies(df_validation, columns=['month'], drop_first=True)

In [21]:
# Encode categorical variables
df_train_merged['Country_idx'] = df_train_merged['Country'].astype('category').cat.codes
df_train_merged['Product_idx'] = df_train_merged['Product'].astype('category').cat.codes

In [22]:
df_train_merged.columns

Index(['unique_id', 'ds', 'Quantity', 'Country', 'Product', 'month_2',
       'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
       'month_9', 'month_10', 'month_11', 'month_12', 'Country_idx',
       'Product_idx'],
      dtype='object')

In [23]:
from mlforecast import MLForecast

fcst = MLForecast(
    models =[],
    freq = 'MS',
    lags=[1,12]
    )

df_train_merged = fcst.preprocess(df_train_merged,target_col='Quantity', id_col='unique_id', time_col='ds',static_features= ['Country', 'Product', 'Country_idx', 'Product_idx']).reset_index(drop=True)

In [24]:
df_train_merged['lag1'] = (df_train_merged['lag1'] - df_train_merged['lag1'].mean()) / df_train_merged['lag1'].std()
df_train_merged['lag12'] = (df_train_merged['lag12'] - df_train_merged['lag12'].mean()) / df_train_merged['lag12'].std()


In [25]:
# Coordinates
coords = {
    #"obs_id": df_train_merged.index,
    "month": [col for col in df_train_merged.columns if col.startswith("month_")],
    "lag": ["lag1", "lag12"],
    "Country": df_train_merged['Country'].astype('category').cat.categories.tolist(),
    #"Product": df_train_merged['Product'].astype('category').cat.categories.tolist(),
}


with pm.Model(coords=coords) as model:
    
    # Mutable shared data containers
    pm_data_month = pm.Data("X_month", df_train_merged[coords["month"]].values)
    pm_data_lags = pm.Data("X_lags", df_train_merged[["lag1", "lag12"]].values)
    pm_data_country = pm.Data("country_idx", df_train_merged["Country_idx"].values)
    pm_data_item = pm.Data("item_idx", df_train_merged["Product_idx"].values)
    
    # Global intercept
    intercept = pm.Normal("intercept", mu=0, sigma=2)
    
    # Month coefficients
    beta_month = pm.Normal("beta_month", mu=0, sigma=1, dims="month")
    
    # Lag coefficients
    beta_lag = pm.Normal("beta_lag", mu=0, sigma=1, dims="lag")
    
    # Country/item intercepts
    country_offset = pm.Normal("country_offset", mu=0, sigma=1, dims="Country")
    
    # Linear predictor
    mu = (
        intercept
        + pm.math.dot(pm_data_month, beta_month) \
        + pm.math.dot(pm_data_lags, beta_lag) \
        + country_offset[pm_data_country] 
        )
    
    # Poisson likelihood
    y = pm.Poisson("y", mu=pm.math.exp(mu), observed=df_train_merged["Quantity"].values)#, dims="obs_id")
    

In [ ]:
with model:
    trace = pm.sample(1000, tune=1000, target_accept=0.9,cores=8,chains=4,return_inferencedata=True)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept, beta_month, beta_lag, country_offset]


Output()

MCMC troppo lento e non tutte le catene divergono!